In [3]:
!git clone https://github.com/i3130002/Tortoise-TTS.git

Cloning into 'Tortoise-TTS'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 13 (delta 1), reused 10 (delta 1), pack-reused 0
Receiving objects: 100% (13/13), 5.44 MiB | 20.31 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [6]:
! cd Tortoise-TTS/ && chmod +x setup.sh && ./setup.sh

Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1923, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 1923 (delta 71), reused 81 (delta 36), pack-reused 1789
Receiving objects: 100% (1923/1923), 54.16 MiB | 32.12 MiB/s, done.
Resolving deltas: 100% (881/881), done.
Looking in indexes: https://download.pytorch.org/whl/cu118
/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:755: SetuptoolsDeprecationWarning: Invalid dash-separated options
!!

        ********************************************************************************
        Usage of dash-separated 'description-file' will not be supported in future
        versions. Please use the underscore name 'description_file' instead.

        This deprecation is overdue, please update your project and remove deprecated
        calls to avoid build errors in the future.

        See https://setuptools.pypa.io/en/latest/userguide/declarative_config

In [8]:
import sys, os
tts_path = os.path.join(str(os.getcwd()), "Tortoise_TTS/tortoise")
sys.path.append(tts_path)
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/tortoise_tts-3.0.0-py3.10.egg',
 '/usr/local/lib/python3.10/dist-packages/transformers-4.31.0-py3.10.egg',
 '/usr/local/lib/python3.10/dist-packages/Unidecode-1.3.8-py3.10.egg',
 '/usr/local/lib/python3.10/dist-packages/einops-0.7.0-py3.10.egg',
 '/usr/local/lib/python3.10/dist-packages/progressbar-2.5-py3.10.egg',
 '/usr/local/lib/python3.10/dist-packages/rotary_embedding_torch-0.5.3-py3.10.egg',
 '/usr/local/lib/python3.10/dist-packages/beartype-0.17.2-py3.10.egg',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/Tortoise_TTS/tortoise',
 '/content/Tortoise_TTS/tortoise']

In [ ]:
!mkdir models

In [ ]:
%tb
import argparse
from time import time

import torch
import torchaudio


from tortoise.api import TextToSpeech, MODELS_DIR
from tortoise.utils.audio import load_audio, load_voices
from tortoise.utils.text import split_and_recombine_text

import sys, os
models_path_default = os.path.join(str(os.getcwd()), "/models")

parser = argparse.ArgumentParser()
parser.add_argument('--voice', type=str, help='Selects the voice to use for generation. See options in voices/ directory (and add your own!) '
                                              'Use the & character to join two voices together. Use a comma to perform inference on multiple voices.', default='i313002')
parser.add_argument('--output_path', type=str, help='Where to store outputs.', default='.')
parser.add_argument('--output_name', type=str, help='How to name the output file', default='output.wav')
parser.add_argument('--preset', type=str, help='Which voice preset to use.', default='standard')
parser.add_argument('--produce_debug_state', type=bool, help='Whether or not to produce debug_state.pth, which can aid in reproducing problems. Defaults to true.', default=True)
parser.add_argument('--use_deepspeed', type=bool, help='Use deepspeed for speed bump.', default=True)
parser.add_argument('--kv_cache', type=bool, help='If you disable this please wait for a long a time to get the output', default=True)
parser.add_argument('--half', type=bool, help="float16(half) precision inference if True it's faster and take less vram and ram", default=True)
parser.add_argument('--model_dir', type=str, help='Where to find pretrained model checkpoints. Tortoise automatically downloads these to .models, so this'
                                                  'should only be specified if you have custom checkpoints.', default=models_path_default)
parser.add_argument('--regenerate', type=str, help='Comma-separated list of clip numbers to re-generate, or nothing.', default=None)
parser.add_argument('--seed', type=int, help='Random seed which can be used to reproduce results.', default=None)

args = parser.parse_args([])


if torch.backends.mps.is_available():
    args.use_deepspeed = False
tts = TextToSpeech(models_dir=args.model_dir, use_deepspeed=args.use_deepspeed, kv_cache=args.kv_cache, half=args.half)

outpath = args.output_path
outname = args.output_name
selected_voices = args.voice.split(',')
regenerate = args.regenerate
if regenerate is not None:
    regenerate = [int(e) for e in regenerate.split(',')]

# Process text
text =""" It comes packed with umbrel (free Bitcoin node is)

I wanted to suggest filehive, from the ob1 team, but it's not running anymore.

I have not tried agora.

Nextcloud could be used for storage. Or IPFS/filecoin. Ipfs would need a browser plugin, or brave browser setting turned on. There are bridges that will connect from old browsers.
"""

if '|' in text:
    print("Found the '|' character in your text, which I will use as a cue for where to split it up. If this was not"
          "your intent, please remove all '|' characters from the input.")
    texts = text.split('|')
else:
    texts = split_and_recombine_text(text)

seed = int(time()) if args.seed is None else args.seed
for selected_voice in selected_voices:
    voice_outpath = os.path.join(outpath, selected_voice)
    os.makedirs(voice_outpath, exist_ok=True)

    if '&' in selected_voice:
        voice_sel = selected_voice.split('&')
    else:
        voice_sel = [selected_voice]

    voice_samples, conditioning_latents = load_voices(voice_sel)
    all_parts = []
    for j, text in enumerate(texts):
        if regenerate is not None and j not in regenerate:
            all_parts.append(load_audio(os.path.join(voice_outpath, f'{j}.wav'), 24000))
            continue
        gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                                  preset=args.preset, k=args.candidates, use_deterministic_seed=seed)
        if args.candidates == 1:
            audio_ = gen.squeeze(0).cpu()
            torchaudio.save(os.path.join(voice_outpath, f'{j}.wav'), audio_, 24000)
        else:
            candidate_dir = os.path.join(voice_outpath, str(j))
            os.makedirs(candidate_dir, exist_ok=True)
            for k, g in enumerate(gen):
                torchaudio.save(os.path.join(candidate_dir, f'{k}.wav'), g.squeeze(0).cpu(), 24000)
            audio_ = gen[0].squeeze(0).cpu()
        all_parts.append(audio_)

    if args.candidates == 1:
        full_audio = torch.cat(all_parts, dim=-1)
        torchaudio.save(os.path.join(voice_outpath, f"{outname}.wav"), full_audio, 24000)

    if args.produce_debug_state:
        os.makedirs('debug_states', exist_ok=True)
        dbg_state = (seed, texts, voice_samples, conditioning_latents)
        torch.save(dbg_state, f'debug_states/read_debug_{selected_voice}.pth')

    # Combine each candidate's audio clips.
    if args.candidates > 1:
        audio_clips = []
        for candidate in range(args.candidates):
            for line in range(len(texts)):
                wav_file = os.path.join(voice_outpath, str(line), f"{candidate}.wav")
                audio_clips.append(load_audio(wav_file, 24000))
            audio_clips = torch.cat(audio_clips, dim=-1)
            torchaudio.save(os.path.join(voice_outpath, f"{outname}_{candidate:02d}.wav"), audio_clips, 24000)
            audio_clips = []

AttributeError: 'Namespace' object has no attribute 'seed'

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


autoregressive.pth:   0%|          | 0.00/1.72G [00:00<?, ?B/s]